In [1]:
import pandas as pd
from pathlib import Path

# -----------------------------
# Config
# -----------------------------
INPUT_CSV = Path("/Users/mariamorazamora/Desktop/Prototyping/Assignment 1/credit_card_transactions.csv") 
OUTPUT_DIR = Path("data")
OUTPUT_DIR.mkdir(exist_ok=True)

USER_INDEX = 24  # i[24] = 25th user in the sorted unique cc_num list

#Export only the columns used fot the prototype
KEEP_COLS = [
    "trans_date_trans_time",
    "cc_num",
    "category",
    "amt",
    "merchant", 
    "is_fraud"   
]

# -----------------------------
# Load
# -----------------------------
df = pd.read_csv(INPUT_CSV)

#Make sure required columns exist
missing = [c for c in KEEP_COLS if c not in df.columns]
if missing:
    print("⚠️ These columns are missing and will be ignored:", missing)
    KEEP_COLS = [c for c in KEEP_COLS if c in df.columns]

#Pick reproducible user: sorted unique cc_nums
unique_users = sorted(df["cc_num"].unique())
if USER_INDEX >= len(unique_users):
    raise ValueError(f"USER_INDEX={USER_INDEX} out of range. Only {len(unique_users)} unique users.")

user_id = unique_users[USER_INDEX]
df_user = df[df["cc_num"] == user_id].copy()

#Parse datetime + add year/month/day for convenience
df_user["trans_date_trans_time"] = pd.to_datetime(df_user["trans_date_trans_time"])
df_user["year"] = df_user["trans_date_trans_time"].dt.year
df_user["month"] = df_user["trans_date_trans_time"].dt.month
df_user["day"] = df_user["trans_date_trans_time"].dt.day

# Keep only needed columns (+ engineered ones)
export_cols = [c for c in KEEP_COLS if c in df_user.columns] + ["year", "month", "day"]
df_user_small = df_user[export_cols].reset_index(drop=True)

#Save
out_path = OUTPUT_DIR / f"transactions_user_{USER_INDEX}_ccnum_{user_id}.csv"
df_user_small.to_csv(out_path, index=False)

print("✅ Exported user subset:")
print("User index:", USER_INDEX)
print("cc_num:", user_id)
print("Rows:", df_user_small.shape[0])
print("Saved to:", out_path.resolve())

✅ Exported user subset:
User index: 24
cc_num: 567868110212
Rows: 2558
Saved to: /Users/mariamorazamora/Desktop/Prototyping/Assignment 1/prototyping-products/data/transactions_user_24_ccnum_567868110212.csv
